In [1]:
from utils import *
from api_calls import *
import pandas as pd
from time import sleep

In [2]:
from load_data import set_mapping

relaxed: False


# Use ground truth data

## outgoing (we created)

### channel nodes

In [3]:
local_node = '025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'
gt_outgoing_channels = read_json(results_folder + 'closedchannels.json')['channels']
channels = pd.read_csv(level2_folder + 'channel.csv')
funding_txs = read_json(level1_folder + 'blockstream_funding_txs.json')
settlement_txs = read_json(level1_folder + 'blockstream_settlement_txs.json')
funded_address_settlement_txs = read_json(level1_folder + 'funded_address_blockstream_settlement_txs.json')
funding_address_entity = read_json(level1_folder + 'funding_address_entity.json')
settlement_address_entity = read_json(level1_folder + 'settlement_address_entity.json')
input_file1 = results_folder + 'filtered_all_heuristic_2_entity_node_conf.json'
heuristic_2_entity_node = read_json(input_file1, double_int_key=True)[2]
input_file3 = results_folder + 'filtered_all_heuristic_2_node_entity_conf.json'
heuristic_2_node_entity_conf = read_json(input_file3, True)[2]
input_file2 = results_folder + 'filtered_all_heuristic_2_entity_node_conf_old.json'
heuristic_2_entity_node_old = read_json(input_file2, double_int_key=True)[2]
on_chain_heuristics = {och: (True if och != 'none' else False) for och in on_chain_heuristics_list}
funding_address_entity, settlement_address_entity = set_mapping(funding_address_entity, settlement_address_entity, on_chain_heuristics)


In [7]:
gt_outgoing_channel_points = [el['channel_point'] for el in gt_outgoing_channels]

In [8]:
print('GT channels opened and closed:', len(gt_outgoing_channel_points))

GT channels opened and closed: 81


In [9]:
closed_channel_nodes = set()
for closed_channel in gt_outgoing_channels:
    closed_channel_nodes.add(closed_channel['remote_pubkey'])
closed_channel_nodes.add(local_node)

In [10]:
print('GT number of nodes with which we closed a channel:', len(closed_channel_nodes))

GT number of nodes with which we closed a channel: 73


In [11]:
len(channels)

98431

In [12]:
chpoint_n1_n2 = dict()
for r in channels.values:
    channel_point, node1, node2 = r
    if channel_point in gt_outgoing_channel_points:
        chpoint_n1_n2[channel_point] = [node1, node2]

### funding node entity

In [13]:
def add_node_to_entity(node, entity, entity_nodes):
    if entity not in entity_nodes:
        entity_nodes[entity] = set()
    entity_nodes[entity].add(node)
    return entity_nodes

In [14]:
gt_entity_node = dict()
for cp, ns in chpoint_n1_n2.items():
    hsh, out_index = cp.split(':')
    funding_address = funding_txs[hsh]['vin'][0]['prevout']['scriptpubkey_address']
    funding_entity = funding_address_entity[funding_address]
    gt_entity_node = add_node_to_entity(local_node, funding_entity, gt_entity_node)

In [15]:
len(gt_entity_node)

79

In [16]:
set([len(str(k)) for k in gt_entity_node.keys()]) # only entities, no components so far

{9}

In [17]:
for e in gt_entity_node:
    if e in heuristic_2_entity_node:
        print(e, heuristic_2_entity_node[e][0][:10], list(gt_entity_node[e])[0][:10])

701936419 025228840b 025228840b
699910001 025228840b 025228840b
701969681 025228840b 025228840b
701997093 025228840b 025228840b
702012094 025228840b 025228840b
702018206 025228840b 025228840b
702049216 025228840b 025228840b


### settlement node entity

In [18]:
received_coins_nodes = set()
for closed_channel in gt_outgoing_channels:
    funding_hsh, out_index = closed_channel['channel_point'].split(':')
    funded_address = funding_txs[funding_hsh]['vout'][int(out_index)]['scriptpubkey_address']
    stx = funded_address_settlement_txs[funded_address][0]
    if not stx['txid'] == closed_channel['closing_tx_hash']:
        print(stx['txid'])
    else:
        # if there are two outputs, the first is remote and the second is local
        if len(stx['vout']) == 2:
            received_coins_nodes.add(closed_channel['remote_pubkey'])
            remote_settlement_entity = settlement_address_entity[stx['vout'][0]['scriptpubkey_address']]
            local_settlement_entity = settlement_address_entity[stx['vout'][1]['scriptpubkey_address']]
            gt_entity_node = add_node_to_entity(local_node, local_settlement_entity, gt_entity_node)
            gt_entity_node = add_node_to_entity(closed_channel['remote_pubkey'], remote_settlement_entity, gt_entity_node)
        # if there is one output, it is local
        elif len(stx['vout']) == 1:
            local_settlement_entity = settlement_address_entity[stx['vout'][0]['scriptpubkey_address']]
            gt_entity_node = add_node_to_entity(local_node, local_settlement_entity, gt_entity_node)        

In [19]:
print('GT number of nodes that received coins from us:', len(received_coins_nodes))

GT number of nodes that received coins from us: 52


In [20]:
for e in gt_entity_node:
    if e in heuristic_2_entity_node:
        print(e, heuristic_2_entity_node[e][0][:10], list(gt_entity_node[e])[0][:10])

701936419 025228840b 025228840b
699910001 025228840b 025228840b
701969681 025228840b 025228840b
701997093 025228840b 025228840b
702012094 025228840b 025228840b
702018206 025228840b 025228840b
702049216 025228840b 025228840b
700906822 02bb24da3d 02bb24da3d
702032339 03da1c27ca 03da1c27ca
701778637 02de11c748 02de11c748
701999821 03fb822818 03fb822818
702049215 03bc9337c7 03bc9337c7


In [21]:
set([len(str(k)) for k in gt_entity_node.keys()]) # only entities, no components so far

{9}

In [22]:
set([len(str(k)) for k in settlement_address_entity.values()]) # entities and components

{3, 4, 5, 7, 8, 9}

In [23]:
len(gt_entity_node)

143

## incoming (they created)

In [24]:
incoming_channels_df = pd.read_csv(level2_folder + 'inbound_channels.csv')

In [25]:
incoming_channels_df.chan_point.values

array(['b65ee1be8c2f98fb38985e8264df1fdcff18950f0bb59079293bf18793953d7d:1',
       '006a68db9594a90783f51a57123d1152164af75c1cda1ca3864d16a7b74d19d9:1',
       'ada40f3b3a5f3d18d36c1be89274d3f2f1316527887cae4ff7284602fef1e979:0',
       '1774fe9c71370d665e89d859fb4cbeca9d9cfebdab1da8ba744d7f7aea545257:0',
       '1331f641fd1dcc1ad6cd6fa3d3c997dc3ea0799ac1820dfad26b3c642a378197:0',
       'a64b0f11e7163e4db9bdedbbf362cf32cd5f3c15512bd910256d722065cadc2c:1'],
      dtype=object)

In [27]:
available_funding_txs = [
    'b65ee1be8c2f98fb38985e8264df1fdcff18950f0bb59079293bf18793953d7d',
    '006a68db9594a90783f51a57123d1152164af75c1cda1ca3864d16a7b74d19d9',
    'ada40f3b3a5f3d18d36c1be89274d3f2f1316527887cae4ff7284602fef1e979',
] 

In [28]:
incoming_channels_df

,chan_point,external_node_key,external_alias
0,b65ee1be8c2f98fb38985e8264df1fdcff18950f0bb590...,03bc9337c7a28bb784d67742ebedd30a93bacdf7e4ca16...,LNBIG.com [lnd-06]
1,006a68db9594a90783f51a57123d1152164af75c1cda1c...,024d2387409269f3b79e2708bb39b895c9f4b6a8322153...,LNBIG.com [lnd-38]
2,ada40f3b3a5f3d18d36c1be89274d3f2f1316527887cae...,030c3f19d742ca294a55c00376b3b355c3c90d61c6b6b3...,Bitrefill.com/gift-cards
3,1774fe9c71370d665e89d859fb4cbeca9d9cfebdab1da8...,0242a4ae0c5bef18048fbecf995094b74bfb0f7391418d...,CoinGate
4,1331f641fd1dcc1ad6cd6fa3d3c997dc3ea0799ac1820d...,03c436af41160a355fc1ed230a64f6a64bcbd2ae50f121...,mainnet.lightningconductor.net
5,a64b0f11e7163e4db9bdedbbf362cf32cd5f3c15512bd9...,03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e...,LightningTo.Me


In [29]:
# external_node_key is linked to the funding entity
nodes_opened_channels_to_us = set()
for r in incoming_channels_df.values:
    chan_point, remote_node, remote_alias = r
    hsh, out_index = chan_point.split(':')
    if hsh in funding_txs:
        nodes_opened_channels_to_us.add(remote_node)
        funding_entity = funding_address_entity[funding_txs[hsh]['vin'][0]['prevout']['scriptpubkey_address']]
        gt_entity_node = add_node_to_entity(remote_node, funding_entity, gt_entity_node)

In [30]:
print('GT number of nodes that opened channels to us:', len(nodes_opened_channels_to_us))

GT number of nodes that opened channels to us: 3


In [31]:
set([len(str(k)) for k in gt_entity_node.keys()]) # only entities, no components so far

{4, 9}

In [32]:
len(gt_entity_node)

146

In [33]:
matches = 0
multiple_nodes_in_entity = 0
for e in gt_entity_node:
    if e in heuristic_2_entity_node:
        print(e, heuristic_2_entity_node[e][0][:10], list(gt_entity_node[e])[0][:10])
        matches += 1
        if len(heuristic_2_entity_node[e]) > 1:
            multiple_nodes_in_entity += 1
print('Number of matches:', matches)
print('Number of cases with multiple nodes linked to one entity :', multiple_nodes_in_entity)

701936419 025228840b 025228840b
699910001 025228840b 025228840b
701969681 025228840b 025228840b
701997093 025228840b 025228840b
702012094 025228840b 025228840b
702018206 025228840b 025228840b
702049216 025228840b 025228840b
700906822 02bb24da3d 02bb24da3d
702032339 03da1c27ca 03da1c27ca
701778637 02de11c748 02de11c748
701999821 03fb822818 03fb822818
702049215 03bc9337c7 03bc9337c7
-432 024d238740 024d238740
Number of matches: 13
Number of cases with multiple nodes linked to one entity : 0


# node to entity

In [34]:
gt_node_entity = dict()
for e, ns in gt_entity_node.items():
    for n in ns:
        gt_node_entity = add_node_to_entity(e, n, gt_node_entity) # don't be fooled by the name ;)

In [35]:
len(gt_node_entity)

55

In [36]:
n_matching_entities = 0
for n in gt_node_entity:
    if n in heuristic_2_node_entity_conf:
        for e in gt_node_entity[n]:
            if e in heuristic_2_node_entity_conf[n]:
                n_matching_entities += 1
    else:
        print(n, 'not linked')
print('Number of matching entities:', n_matching_entities)

Number of matching entities: 13


In [37]:
nodes = pd.read_csv(level2_folder + 'node.csv')

In [38]:
node_aliases = dict()
for r in nodes.values:
    node, aliases = r
    aliases = set(aliases.split('|'))
    if node not in node_aliases:
        node_aliases[node] = set()
    node_aliases[node] = node_aliases[node].union(aliases)

In [39]:
validated_nodes = set()
for n in gt_node_entity:
    if n in heuristic_2_node_entity_conf:
#         print('Node:', n)
        alias = ''
        if n in node_aliases:
            alias = node_aliases[n]
#         print('Alias:', alias)
        gt_entities = list(set(gt_node_entity[n]))
        linked_entities = list(set(heuristic_2_node_entity_conf[n]))
        gt_entities.sort()
        linked_entities.sort()
#         print('Number of ground truth entities:', len(gt_entities))
#         for e in gt_entities:
#             print(e, end=' ')
#         print()
#         print('Number of linked entities:', len(linked_entities))
#         for e in linked_entities:
#             print(e, end=' ')
#         print()
        len_intersection = len(set(linked_entities).intersection(set(gt_entities)))
#         print('Intersection:', len_intersection)
        if len_intersection:
            validated_nodes.add(n)
#     else:
#         print(n, 'not linked')
#     print()

In [40]:
print('Number of validated nodes:', len(validated_nodes))

Number of validated nodes: 7


In [41]:
linked_nodes = set()
for ns in heuristic_2_entity_node.values():
    for n in ns:
        linked_nodes.add(n)

In [42]:
len(linked_nodes)

4600

# find indirect connection between gt entity and linked entity for unvalidated nodes

## find the minimum distance between the gt entity and the linked entities

In [72]:
# for node in gt_node_entity:
#     if node not in validated_nodes:
#         print('Node:', node)
#         print('Alias:', node_aliases[node])
#         print('GT entities:', gt_node_entity[node])
#         print('Linked entities:', heuristic_2_node_entity_conf[node])
#         print()

In [41]:
def get_entity_neighbors(entity, currency='btc'):
    nbrs_out = get_entity_neighbors_out(entity, currency)
    nbrs_in = get_entity_neighbors_in(entity, currency)
    return nbrs_in.union(nbrs_out)

def get_entity_neighbors_out(entity, currency='btc'):
    # get all entities receiving money from this entity
    neighbors_out = set()
    url = api_path + currency + '/' + 'entities/' + str(entity) + '/neighbors?direction=out'
    d = url2dict(url)
    if 'neighbors' in d:
        for n in d['neighbors']:
            neighbors_out.add(int(n['id']))
    return neighbors_out

def get_entity_neighbors_in(entity, currency='btc'):
    # get all entities sending money to this entity
    neighbors_in = set()
    url = api_path + currency + '/' + 'entities/' + str(entity) + '/neighbors?direction=in'
    d = url2dict(url)
    if 'neighbors' in d:
        for n in d['neighbors']:
            neighbors_in.add(int(n['id']))
    return neighbors_in

In [42]:
gt_entity_hop_nbrs = dict() # key: gt_entity, value: dict of key: hop, value: neighbors

In [43]:
# h = 1
# for n, es in gt_node_entity.items():
#     if n not in validated_nodes:
#         print()
#         print(n)
#         for e in es:
#             if e not in gt_entity_hop_nbrs:
#                 gt_entity_hop_nbrs[e] = dict()
#                 gt_entity_hop_nbrs[e][h] = set()
#             # get neighbors at hop 1
#             if not gt_entity_hop_nbrs[e][h] and e > 0:
#                 print(e)
#                 print('getting nbrs')
#                 gt_entity_hop_nbrs[e][h] = gt_entity_hop_nbrs[e][h].union(get_entity_neighbors(e))

## check if there are nbrs of gt_entity in linked_entities

In [43]:
# for n, es in gt_node_entity.items():
#     if n not in validated_nodes:
#         for e in es:
#             nbrs = gt_entity_hop_nbrs[e][h]
#             nbrs_linked_entities_intersection = nbrs.intersection(heuristic_2_node_entity_conf[n])
#             if nbrs_linked_entities_intersection:
#                 print(e, len(nbrs_linked_entities_intersection), n)
#                 validated_nodes.add(n)

In [43]:
validated_nodes.add('03fce165537aea120bffe8505876b44d5119354f825b3eac329b761fc5636bf334')
validated_nodes.add('0311cad0edf4ac67298805cf4407d94358ca60cd44f2e360856f3b1c088bcd4782')

In [44]:
indirectly_validated_nodes = set()
indirectly_validated_nodes.add('03fce165537aea120bffe8505876b44d5119354f825b3eac329b761fc5636bf334')
indirectly_validated_nodes.add('0311cad0edf4ac67298805cf4407d94358ca60cd44f2e360856f3b1c088bcd4782')

# count number of remote nodes that spent coins from settlements

In [46]:
# for closed_channel in gt_outgoing_channels:
#     funding_hsh, out_index = closed_channel['channel_point'].split(':')
#     funded_address = funding_txs[funding_hsh]['vout'][int(out_index)]['scriptpubkey_address']
#     stx = funded_address_settlement_txs[funded_address][0]
#     # if there are two outputs, the first is remote and the second is local
#     if len(stx['vout']) == 2:
#         print(closed_channel['remote_pubkey'])
#         print('Already validated:', closed_channel['remote_pubkey'] in validated_nodes)
#         print(stx['txid'])
#         print(stx['vout'][0]['scriptpubkey_address'])
#         print()
        

In [45]:
# manually checked,  spent before last_block = 647529
settlement_txs_where_remote_spends_coins =[
    'd6d658c4a13c8f2d2927a71e1cdd5ef310d7d9adb9f96774018276a3590c3788',
    'be051cfb727c10c28c3975d8d32a0c29bb244b4f21bd14e2eab584219f496b27',
    '9fa69e68dc5ce6525b3edeb7dba8f2d954adea3d82ca340a55b7157418d384c1',
    '101c492db10266eb1c7cd63e00bcfbb9f60860a0badff6c7573b673f523b45f2',
    'f65aef03e5a93d5acebf8135cd411ccf46013a957620eab2a5a95171327f4e93',
    'e836d71d6cc8b5a79562b46890429a89ecc5e9e3be8cbc0203a1c00bd69c8d2a',
    'fe9f60f930d1a7cad6b17923c8f3f041b5e2ba308447c73d82abc048389c930a',
    '75f27715d27c6629673ddf080cf5267dc85bea40bf702fcb59377f82553b7e08',
    '2894fac92b98402a993b6b57db0877db085d796978c89495b630871e12b2427f',
    '4dd694546be280a08803b9e2eb9e15adfe0f4e4ef0f53567d9a7f183188ebcff',
    '7e25d41fd47d10287e560c3d98cebe041c0f7ae57c1fff270ad662753ec706c0',
    'f4c30c226bb4ce2c16673555768190318dd327bab9d06b51ff35a497483eff70',
    '399cfcd171e69d7a0c150772bba5202850c0186e58d9b818cfaf6a4c74f567fa',
    '05489ca075ff037934e734b893098559c10eabc953edca1cc2faa80fe1042582',
]

In [46]:
def get_address_txs(address, currency='btc'):
    url = api_path + currency + '/addresses/' + address 
    d = url2dict(url)
    return d

In [ ]:
# node_settlement_address_txs = dict()

In [226]:
# # check activity of node addresses not spending our coins
# nodes_that_spent_our_coins = set()
# for closed_channel in gt_outgoing_channels:
#     funding_hsh, out_index = closed_channel['channel_point'].split(':')
#     funded_address = funding_txs[funding_hsh]['vout'][int(out_index)]['scriptpubkey_address']
#     stx = funded_address_settlement_txs[funded_address][0]
#     # if there are two outputs, the first is remote and the second is local
#     if len(stx['vout']) == 2:
#         if stx['txid'] not in settlement_txs_where_remote_spends_coins:
#             if closed_channel['remote_pubkey'] not in node_settlement_address_txs:
#                 node_settlement_address_txs[closed_channel['remote_pubkey']] = dict()
#             a = stx['vout'][0]['scriptpubkey_address']
#             if a not in node_settlement_address_txs[closed_channel['remote_pubkey']]:
#                 node_settlement_address_txs[closed_channel['remote_pubkey']][a] = get_address_txs(a)
#             elif not node_settlement_address_txs[closed_channel['remote_pubkey']][a]:
#                 node_settlement_address_txs[closed_channel['remote_pubkey']][a] = get_address_txs(a)            
#         else:
#             nodes_that_spent_our_coins.add(closed_channel['remote_pubkey'])
# for node in nodes_that_spent_our_coins:
#     node_settlement_address_txs.pop(node, None)

In [235]:
# write_json(node_settlement_address_txs, results_folder + 'node_settlement_address_txs.json')

In [47]:
node_settlement_address_txs = read_json(results_folder + 'node_settlement_address_txs.json')

In [48]:
print('GT number of nodes that received our coins and did not spend them:', len(node_settlement_address_txs.keys()))

GT number of nodes that received our coins and did not spend them: 41


In [49]:
print('GT number of addresses that received our coins and did not spend them', sum([len(d.keys()) for d in node_settlement_address_txs.values()]))

GT number of addresses that received our coins and did not spend them 43


In [50]:
node_address_no_spend = set()
# the node receives coins on addresses that have only 1 incoming tx
for node, d in node_settlement_address_txs.items():
    discard = False # discard if address is not new or spent coins
    for a, txs in d.items():
        # discard if the address has more than 1 incoming tx or spent 
        if txs['no_incoming_txs'] > 1 or txs['no_outgoing_txs']: 
            discard = True
    if not discard:
        node_address_no_spend.add(node)

In [51]:
print('GT number of nodes that did not reuse addresses, received our coins and never spent any coins:', len(node_address_no_spend))

GT number of nodes that did not reuse addresses, received our coins and never spent any coins: 41


In [52]:
received_spent_coins_nodes = set()
for closed_channel in gt_outgoing_channels:
    funding_hsh, out_index = closed_channel['channel_point'].split(':')
    funded_address = funding_txs[funding_hsh]['vout'][int(out_index)]['scriptpubkey_address']
    stx = funded_address_settlement_txs[funded_address][0]
    # if there are two outputs, the first is remote and the second is local
    if len(stx['vout']) == 2 and stx['txid'] in settlement_txs_where_remote_spends_coins:
        received_spent_coins_nodes.add(closed_channel['remote_pubkey'])

In [53]:
print('GT number of nodes that received and spent our coins:', len(received_spent_coins_nodes))

GT number of nodes that received and spent our coins: 11


In [54]:
print('GT number of nodes that received and spent our coins and are validated:', len(received_spent_coins_nodes.intersection(validated_nodes)))

GT number of nodes that received and spent our coins and are validated: 7


In [55]:
received_coins_linked_nodes = received_coins_nodes.intersection(set(heuristic_2_node_entity_conf.keys()))

In [56]:
print('GT number of nodes that received our coins and were linked with our heuristic:', len(received_coins_linked_nodes))

GT number of nodes that received our coins and were linked with our heuristic: 52


In [57]:
print('Incoming')
print('Number of nodes that opened channels to us:', len(nodes_opened_channels_to_us))
print('Number of nodes that opened channels to us and are validated:', len(nodes_opened_channels_to_us.intersection(validated_nodes)))
print()
print('Outgoing')
print('Number of channels we opened and closed:', len(gt_outgoing_channel_points))
print('Number of nodes with which we closed a channel:', len(closed_channel_nodes))
print('Number of nodes that received coins from us:', len(received_coins_nodes))
print('Number of nodes that received and spent our coins:', len(received_spent_coins_nodes))
print('Number of nodes that received and spent our coins and are validated:', len(received_spent_coins_nodes.intersection(validated_nodes)))
print('Number of nodes that received our coins and were linked with our heuristic:', len(received_coins_linked_nodes))
print('Number of nodes that received our coins using new addresses which never spent any coins:', len(node_address_no_spend))
print()
print('Total')
print('Number of validated nodes:', len(validated_nodes))



Incoming
Number of nodes that opened channels to us: 3
Number of nodes that opened channels to us and are validated: 2

Outgoing
Number of channels we opened and closed: 81
Number of nodes with which we closed a channel: 73
Number of nodes that received coins from us: 52
Number of nodes that received and spent our coins: 11
Number of nodes that received and spent our coins and are validated: 7
Number of nodes that received our coins and were linked with our heuristic: 52
Number of nodes that received our coins using new addresses which never spent any coins: 41

Total
Number of validated nodes: 9


In [60]:
print('nNodesOpenedChannelsToUs:', len(nodes_opened_channels_to_us))
print('nNodesOpenedChannelsToUsValidated:', len(nodes_opened_channels_to_us.intersection(validated_nodes)))
print('nChannelsWeOpenedAndClosed:', len(gt_outgoing_channel_points))
print('nNodesWeClosedChannelWith:', len(closed_channel_nodes))
print('nNodesReceivedOurCoins:', len(received_coins_nodes))
print('nNodesReceivedSpentOurCoins:', len(received_spent_coins_nodes))
print('nNodesReceivedSpentOurCoinsValidates:', len(received_spent_coins_nodes.intersection(validated_nodes)))
print('nNodesReceivedOurCoinsLinked:', len(received_coins_linked_nodes))
print('nNodesReceivedOurCoinsNewAddressUnspent:', len(node_address_no_spend))
print('nNodesValidated:', len(validated_nodes))



nNodesOpenedChannelsToUs: 3
nNodesOpenedChannelsToUsValidated: 2
nChannelsWeOpenedAndClosed: 81
nNodesWeClosedChannelWith: 73
nNodesReceivedOurCoins: 52
nNodesReceivedSpentOurCoins: 11
nNodesReceivedSpentOurCoinsValidates: 7
nNodesReceivedOurCoinsLinked: 52
nNodesReceivedOurCoinsNewAddressUnspent: 41
nNodesValidated: 9


In [62]:
n_stxs_two_outputs = 0
for ct in gt_outgoing_channels:
    hsh = ct['closing_tx_hash']
    l = len(settlement_txs[hsh]['vout'])
    if l == 2:
        n_stxs_two_outputs += 1
print('GT number of settlement txs with two outputs', n_stxs_two_outputs)

58

In [63]:
n_stxs_two_outputs = 0
for ct in gt_outgoing_channels:
    hsh = ct['closing_tx_hash']
    l = len(settlement_txs[hsh]['vout'])
    if l == 2:
        n_stxs_two_outputs += 1
print('nSettlementTxsWithTwoOutputs', n_stxs_two_outputs)

nSettlementTxsWithTwoOutputs 58


In [65]:
len(gt_entity_node)

146

In [66]:
gt_entity_node

{702942679: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702942707: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702942677: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702942754: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702943095: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702942676: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702942672: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702942682: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702942683: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702942671: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702649586: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'},
 702801206: {'025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa

### missing validations

In [78]:
for n in received_spent_coins_nodes:
    if n not in validated_nodes:
        print(n)
        print(node_aliases[n])
        print(gt_node_entity[n])
        print(len(heuristic_2_node_entity_conf[n]))
        print()

0303a518845db99994783f606e6629e705cfaf072e5ce9a4d8bf9e249de4fbd019
{'LNBIG.com [lnd-25]'}
{702145255}
83

03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f
{'ACINQ'}
{700855070}
375

031ce29116eab7edd66148f5169f1fb658fad62bdc5091221ab895fe5d36db00b2
{'LNBIG.com [lnd-05]'}
{701940461}
93

032d4baebebfdeab7a2ecef2fbe109cbef10de95f05aa54090fdb687789547dbf5
{'CONNECT_WITH_ME'}
{702410217}
25

